## Components RAG Chain

- Context: vectordb
- Prompt: question
- LLM: model to run the query with
- OutputParser: output format

```python
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
```

### Instantiate vectordb

In [6]:
from modules.vectordb import get_vectordb_bocyl
vectordb = get_vectordb_bocyl()

### Vectordb as retriever

In [7]:
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

### Template for the context

In [8]:
template = """Answer the question based only on the following context:

{context}

Question: {question}

Answer in Spanish. If the answer cannot be found in the context, say "No encuentro información sobre esto en los documentos proporcionados."
"""

In [9]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n\n{context}\n\nQuestion: {question}\n\nAnswer in Spanish. If the answer cannot be found in the context, say "No encuentro información sobre esto en los documentos proporcionados."\n'), additional_kwargs={})])

### Instantiate LLM

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="chatgpt-4o-latest")

### Compose RAG Chain

In [11]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Test the RAG Chain


### Query

In [12]:
query = "¿Qué documentos hay sobre condicionalidad reforzada?"
response = rag_chain.invoke(query)
print(response)

En los documentos proporcionados, se mencionan varios documentos relacionados con la condicionalidad reforzada:

1. Una memoria de actuación justificativa de la realización del proyecto y del cumplimiento de condiciones impuestas en la concesión de la subvención, con indicación de las actividades realizadas y de los resultados obtenidos.

2. Una memoria económica justificativa del coste de las actividades realizadas, que contendrá:
   a. Una relación clasificada de los gastos de la actividad realizada.
   b. Las facturas o documentos de valor probatorio equivalente y la documentación acreditativa del pago.

Además, se prevé un «Plan de controles administrativos y sobre el terreno para la comprobación de los requisitos de la condicionalidad reforzada y el cálculo de las penalizaciones».

También se establece que las áreas competentes en materia de condicionalidad reforzada proporcionarán datos e información a la Consejería competente en materia agraria.


### Save outputs

In [13]:
import datetime
import hashlib

# Generate the current datetime string
current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Generate a hash of the response
response_hash = hashlib.md5(response.encode()).hexdigest()

# Create the filename with datetime and hash
filename = f"outputs/{current_datetime}_{response_hash}.md"

# Save the response to the file
with open(filename, 'w') as f:
    f.write(response)